<a href="https://colab.research.google.com/github/andyradz/aplios.solutions/blob/master/DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataFame

In [ ]:
! pyspark

In [2]:
! pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=caff527bd4055da226714b03888fd8e91cf67b60e360ed5c2e3dab1e8f6cc40b
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [5]:
! wget https://drive.google.com/uc?id=1G9H9BFtcQrokludqfvDv8sz58DwT7Mn6&export=download -O data.zip

/bin/bash: -O: command not found
--2023-04-14 08:01:22--  https://drive.google.com/uc?id=1G9H9BFtcQrokludqfvDv8sz58DwT7Mn6
Resolving drive.google.com (drive.google.com)... 173.194.192.138, 173.194.192.102, 173.194.192.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.192.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-1o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lhmdckcroh1mlourghlq3cpaqhi55jpj/1681459275000/05017911732949556247/*/1G9H9BFtcQrokludqfvDv8sz58DwT7Mn6?uuid=c5d2f89e-0301-4823-b64a-36b7c0d27bc4 [following]
--2023-04-14 08:01:27--  https://doc-10-1o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lhmdckcroh1mlourghlq3cpaqhi55jpj/1681459275000/05017911732949556247/*/1G9H9BFtcQrokludqfvDv8sz58DwT7Mn6?uuid=c5d2f89e-0301-4823-b64a-36b7c0d27bc4
Resolving doc-10-1o-docs.googleusercontent.com (doc-10-1o-docs.googleusercontent.com)... 173.194.198.132, 2607:

In [7]:
! 7z x data.zip -o* -aoa


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 25234788 bytes (25 MiB)

Extracting archive: data.zip
--
Path = data.zip
Type = zip
Physical Size = 25234788

  0%     18% - 2017-fordgobike-tripdata.csv                                    39% - 2017-fordgobike-tripdata.csv                                    62% - 2017-fordgobike-tripdata.csv                                    79% 4 - Badges.xml                    99% 8 - postgresql-42.2.18.jar                               

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [22]:
! pip freeze | grep -i spark

pyspark==3.4.0


https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html

In [69]:
from pyspark.sql import SparkSession#, SparkConf
from pyspark import SparkConf
from pyspark.sql import Row
from pyspark.sql import functions as f

conf = SparkConf()
conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
#.conf(conf) \

spark = SparkSession.builder \
    .appName('app') \
    .getOrCreate()

## Tworzenie DataFrame'u

### Kolekcja Row'ów

In [23]:
person1 = Row(age=32, name='Greg')

In [24]:
person1

Row(age=32, name='Greg')

In [25]:
person1.name, person1.age

('Greg', 32)

In [26]:
person1['age']

32

In [28]:
person1[1]

'Greg'

In [27]:
'age' in person1

True

In [29]:
newPerson = Row("age", "name")

In [30]:
person2 = newPerson(24, 'Alice')

In [31]:
person2

Row(age=24, name='Alice')

In [32]:
person3 = newPerson(None, None)
person4 = newPerson(33, None)
person5 = newPerson(None, 'Peter')
person6 = newPerson(32, 'Peter')
person7 = newPerson(40, 'Greg')

In [33]:
peopleDF = spark.createDataFrame([person1, person2, person3, person4, 
                                  person5, person6, person7])

In [34]:
peopleDF.show()

+----+-----+
| age| name|
+----+-----+
|  32| Greg|
|  24|Alice|
|null| null|
|  33| null|
|null|Peter|
|  32|Peter|
|  40| Greg|
+----+-----+



### Inne lokalne kolekcje

Typy danych: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#data-types

Kilka podstawowych: IntegerType, DoubleType, FloatType, StringType, BooleanType, NullType

In [35]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField

In [36]:
# definicja schematu
# StructType ~ Row
schema = StructType([StructField("V1", IntegerType()), StructField("V2", StringType())])

In [44]:
# lokalna kolekcja - lista list
df = spark.createDataFrame([(1,2),(3,4)], schema)

In [41]:
df.show()
df.printSchema()

+---+---+
| V1| V2|
+---+---+
|  1|  2|
|  3|  4|
+---+---+

root
 |-- V1: integer (nullable = true)
 |-- V2: string (nullable = true)



### RDD

Przechodzenie RDD <-> DF

In [45]:
type(peopleDF)

pyspark.sql.dataframe.DataFrame

In [46]:
peopleDF.show()

+----+-----+
| age| name|
+----+-----+
|  32| Greg|
|  24|Alice|
|null| null|
|  33| null|
|null|Peter|
|  32|Peter|
|  40| Greg|
+----+-----+



DF -> RDD

In [47]:
peopleRDD = peopleDF.rdd
type(peopleRDD)

pyspark.rdd.RDD

In [48]:
peopleRDD.collect()

[Row(age=32, name='Greg'),
 Row(age=24, name='Alice'),
 Row(age=None, name=None),
 Row(age=33, name=None),
 Row(age=None, name='Peter'),
 Row(age=32, name='Peter'),
 Row(age=40, name='Greg')]

In [49]:
peopleRDD.map(tuple).mapValues(lambda x: "X" if x is None else x).collect()

[(32, 'Greg'),
 (24, 'Alice'),
 (None, 'X'),
 (33, 'X'),
 (None, 'Peter'),
 (32, 'Peter'),
 (40, 'Greg')]

RDD -> DF

In [ ]:
peopleRDD.toDF().show()

In [ ]:
peopleRDD.map(tuple).toDF().show()

In [ ]:
peopleRDD.map(tuple).toDF().dtypes

In [ ]:
sc = spark.sparkContext

In [ ]:
tupRDD = sc.parallelize([(x, x+1) for x in range(5)])
tupRDD.toDF().show()

In [ ]:
# do toDF można podać schemat
schema = StructType([StructField("A", IntegerType()), StructField("B", StringType())])
tupRDD.toDF(schema).show()

> **TODO**: Stwórz DF z 3 wierszami i 3 kolumnami - dwie typu string, jedna numeryczna

In [55]:
schema = StructType([StructField("Name", StringType()), StructField("Sureame", StringType()), StructField("Age", IntegerType())])

In [56]:
personsDF=spark.createDataFrame([('Andrzej', 'R', 12),('Hubert', 'W', 34),('Marcin', 'P', 34)], schema)
personsDF.show()
personsDF.printSchema()


+-------+-------+---+
|   Name|Sureame|Age|
+-------+-------+---+
|Andrzej|      R| 12|
| Hubert|      W| 34|
| Marcin|      P| 34|
+-------+-------+---+

root
 |-- Name: string (nullable = true)
 |-- Sureame: string (nullable = true)
 |-- Age: integer (nullable = true)



### Plik

Obsługiwane: text, csv, [nd]json, orc, parquet, avro

In [57]:
empHist = spark.read.parquet("data/salary_hist")

In [58]:
empHist.show()

+-----+-------+----+-------+
| name| salary|year|company|
+-----+-------+----+-------+
|Alice|4404.23|2005|      X|
|Alice| 3000.0|2005|      Y|
|Alice|4780.34|2006|      X|
|Alice|4881.72|2007|      X|
|Alice|5280.86|2008|      X|
|Alice|5976.68|2009|      Z|
|Alice|6320.14|2010|      Z|
|Betty|4138.01|2005|      Y|
|Betty|4376.94|2006|      Y|
|Betty|5117.68|2007|      X|
|Betty|5630.26|2008|      X|
|Betty|6222.57|2009|      X|
|Betty|6623.97|2010|      X|
|Chris|3601.42|2005|      X|
|Chris|4015.66|2006|      X|
|Chris|4304.73|2007|      X|
|Chris|4650.33|2008|      X|
|Chris|4932.86|2009|      X|
|Chris|5869.98|2010|      X|
|  Dan|4262.75|2005|      Y|
+-----+-------+----+-------+
only showing top 20 rows



In [59]:
empHist.explain()

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [name#280,salary#281,year#282,company#283] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/data/salary_hist], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<name:string,salary:double,year:int,company:string>




In [63]:
empHist.select("name").explain()

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [name#280] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/data/salary_hist], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<name:string>




Źródło danych: https://archive.ics.uci.edu/ml/datasets/adult

In [64]:
! head data/adult.data

39, State-gov, 77516, Bachelors, 13, Never-married, Adm-clerical, Not-in-family, White, Male, 2174, 0, 40, United-States, <=50K
50, Self-emp-not-inc, 83311, Bachelors, 13, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0, 0, 13, United-States, <=50K
38, Private, 215646, HS-grad, 9, Divorced, Handlers-cleaners, Not-in-family, White, Male, 0, 0, 40, United-States, <=50K
53, Private, 234721, 11th, 7, Married-civ-spouse, Handlers-cleaners, Husband, Black, Male, 0, 0, 40, United-States, <=50K
28, Private, 338409, Bachelors, 13, Married-civ-spouse, Prof-specialty, Wife, Black, Female, 0, 0, 40, Cuba, <=50K
37, Private, 284582, Masters, 14, Married-civ-spouse, Exec-managerial, Wife, White, Female, 0, 0, 40, United-States, <=50K
49, Private, 160187, 9th, 5, Married-spouse-absent, Other-service, Not-in-family, Black, Female, 0, 0, 16, Jamaica, <=50K
52, Self-emp-not-inc, 209642, HS-grad, 9, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0, 0, 45, United-States, >50K
31, 

In [14]:
# csv
adultDF = spark.read.csv("data/adult.data", inferSchema=True, ignoreLeadingWhiteSpace=True)

In [15]:
adultDF.show()

+---+----------------+------+------------+---+--------------------+-----------------+-------------+------------------+------+-----+----+----+-------------+-----+
|_c0|             _c1|   _c2|         _c3|_c4|                 _c5|              _c6|          _c7|               _c8|   _c9| _c10|_c11|_c12|         _c13| _c14|
+---+----------------+------+------------+---+--------------------+-----------------+-------------+------------------+------+-----+----+----+-------------+-----+
| 39|       State-gov| 77516|   Bachelors| 13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male| 2174|   0|  40|United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors| 13|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Male|    0|   0|  13|United-States|<=50K|
| 38|         Private|215646|     HS-grad|  9|            Divorced|Handlers-cleaners|Not-in-family|             White|  Male|    0|   0|  40|United-States|<=50K|
| 53|         Private|234721

In [70]:
adultDF.take(1)

[Row(_c0=39, _c1='State-gov', _c2=77516, _c3='Bachelors', _c4=13, _c5='Never-married', _c6='Adm-clerical', _c7='Not-in-family', _c8='White', _c9='Male', _c10=2174, _c11=0, _c12=40, _c13='United-States', _c14='<=50K')]

In [71]:
col_names = ["age", "workclass", "fnlwgt", "education", "education-num","marital-status", "occupation", 
             "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", 
             "native-country", "earnings"]

In [72]:
adultDF = adultDF.toDF(*col_names).drop("fnlwgt").dropna("any")

In [73]:
adultDF.show(3, vertical=True)

-RECORD 0----------------------------
 age            | 39                 
 workclass      | State-gov          
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Never-married      
 occupation     | Adm-clerical       
 relationship   | Not-in-family      
 race           | White              
 sex            | Male               
 capital-gain   | 2174               
 capital-loss   | 0                  
 hours-per-week | 40                 
 native-country | United-States      
 earnings       | <=50K              
-RECORD 1----------------------------
 age            | 50                 
 workclass      | Self-emp-not-inc   
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Married-civ-spouse 
 occupation     | Exec-managerial    
 relationship   | Husband            
 race           | White              
 sex            | Male               
 capital-gain   | 0                  
 capital-los

## Zapisywanie DataFrame'u

In [ ]:
#peopleDF.write.csv("...")

In [ ]:
#peopleDF.write.parquet("...")

**Partycjonowanie** - tworzenie podkatalogów w katalogu z danymi w oparciu o unikatowe wartości we wskazanej kolumnie/kolumnach - zapytania dotyczące tylko jednego z poziomów zmiennej na podstwie której dokonano partycjonowania będą musiały przeskanować jedynie fragment danych a nie całość

**Bucketing** - sposób na rozdystrybuowanie/organizację danych na wskazaną (stałą) liczbę buketów, każdy bucket to osobny plik, przypisanie rekordu do bucketu wynika z hasha wartości wskazanej kolumny/kolumn

In [77]:
adultDF.write\
.mode("overwrite")\
.partitionBy("earnings")\
.parquet("adult_partition")

In [78]:
adultDF.write\
.format("parquet")\
.partitionBy("earnings")\
.saveAsTable("adult_partition")

In [81]:
adultDF.write\
.mode("overwrite")\
.format("parquet")\
.bucketBy(4, "workclass", "education", "occupation")\
.saveAsTable("adult_bucket")

In [82]:
spark.sql("select age from adult_bucket limit 10").show()

+---+
|age|
+---+
| 50|
| 28|
| 52|
| 31|
| 37|
| 23|
| 32|
| 25|
| 32|
| 54|
+---+



****

## Praca z DataFrame'ami

### Kolumny

Odwolania do poszczegolnych kolumn

In [83]:
peopleDF.age

Column<'age'>

In [84]:
peopleDF['age']

Column<'age'>

In [ ]:
peopleDF[0]

In [85]:
import 
f.col("age")
srednia = f.col("age")
peopleDF.select(average(srednia)).show()

+----+
| age|
+----+
|  32|
|  24|
|null|
|  33|
|null|
|  32|
|  40|
+----+



In [ ]:
"age"

Lista kolumn 

In [ ]:
peopleDF.columns

In [ ]:
peopleDF.dtypes

Schemat

In [ ]:
peopleDF.printSchema()

### Rejestracja DF jako temp view

In [ ]:
empHist.createOrReplaceTempView("empHist")
adultDF.createOrReplaceTempView('adultDF')
peopleDF.createOrReplaceTempView('peopleDF')

In [ ]:
spark.sql("select * from empHist limit 10").show()

In [ ]:
spark.sql("select age, workclass from adultDF limit 10").show()

In [ ]:
spark.sql("select * from peopleDF").show()

In [ ]:
# usunięcie temp view
#spark.catalog.dropTempView("peopleDF")

### Przegląd akcji i transformacji

----
- **filter(cond) / where(cond)** - zwraca nowy DF z wierszami spełniającymi wskazany warunek (**T**)
- **select(cols)** - zwraca nowy DF ze wskazanymi kolumnami (**T**)
- **distinct()** - zwraca nowy DF z unikalnymi wierszami z oryginalnego DF (**T**)
- **show(n, truncate, vertical)** - wyświetla `n` wierszy DF (**A**)

In [ ]:
empHist \
.where((empHist.company == "X") & (empHist.salary > 6000)) \
.select("name") \
.distinct() \
.show()

In [ ]:
spark.sql("select distinct name from empHist where company = 'X' and salary > 6000").show()

In [ ]:
empHist \
.where((empHist.company == "X") & (empHist.salary > 6000)) \
.select("name") \
.distinct() \
.explain()

In [ ]:
spark.sql("select distinct name from empHist where company = 'X' and salary > 6000").explain()

----
- **withColumn(name, col)** - zwraca nowy DF zawierający nowo zdefiniowaną kolumnę (**T**)
- **drop(cols)** - zwraca nowy DF bez wskazanych kolumn (**T**)
- **withColumnRenamed(old, new)** - zwraca nowy DF ze zmienioną nazwą kolumny (**T**)

In [ ]:
empHist \
.withColumn("salary", empHist.salary * 0.8) \
.drop("company") \
.withColumnRenamed("name", "imie") \
.show()

In [ ]:
empHist \
.select(empHist["name"].alias("imie"), (empHist["salary"] * 0.8).alias("salary"), "year") \
.show()

In [ ]:
spark.sql("select name imie, salary * 0.8 salary, year from empHist").show()

----
- **collect()** - zwraca elementy DF na driver (**A**)

In [ ]:
peopleDF \
.filter(peopleDF.age.isNotNull() | peopleDF.name.contains("t")) \
.collect()

In [ ]:
peopleDF \
.filter(peopleDF.age.isNotNull() | peopleDF.name.like("%t%")) \
.collect()

In [ ]:
spark.sql("select age, name from peopleDF where age is not null or name like '%t%'").collect()

----
- **dropna(how, thresh, subset)** - zwraca nowy DF z usuniętymi wierszami zawierającymi braki danych (**T**)
- **orderBy(cols, ascending)** - zwraca nowy DF z wierszami posortowanymi według wartości kolumn(y) (**T**)

In [ ]:
peopleDF \
.dropna("all") \
.orderBy("age") \
.show()

In [ ]:
spark.sql("select age, name from peopleDF where age is not null or name is not null order by age").show()

In [ ]:
empHist.orderBy(["name", "salary"], ascending=[0,1]).show()

----
- **fillna(value, subset)** - zwraca nowy DF z brakami danych zastąpionymi zdefiniowaną wartością/wartościami (**T**)
- **take(n) / head(n)** - zwraca piersze `n` wierszy z DF jako lokalną kolekcję (listę) wierszy (**A**)

In [ ]:
peopleDF \
.fillna("X") \
.head(7)

In [ ]:
peopleDF \
.fillna({"age": 0, "name": "X"}) \
.take(7)

In [ ]:
peopleDF \
.select(f.when(peopleDF.age.isNull(),0).otherwise(peopleDF.age).alias("age"), 
        f.when(peopleDF.name.isNull(),"X").otherwise(peopleDF.name).alias("name")) \
.show()

In [ ]:
q = ("select case when isnull(age) then 0 else age end age, "
     "case when isnull(name) then 'X' else name end name from peopleDF")
spark.sql(q).show()

----
- **replace(old, new, subset)** - zwraca nowy DF z podmienionymi wartościami (**T**)
- **union(df)** - zwraca nowy DF zawierający wszystkie wiersze z dwóch łączonych DFów (**T**)

In [ ]:
peopleDF \
.replace("Peter", "Piotr") \
.union(peopleDF) \
.show()

In [ ]:
peopleDF \
.replace({"Peter": "Piotr", "Greg": "Grzegorz", "Alice": "Alicja"}) \
.union(peopleDF) \
.show()

In [ ]:
# kolejność ma znaczenie
peopleDF \
.select(f.when(peopleDF.name == "Peter", "Piotr").otherwise(peopleDF.name).alias("name"), "age") \
.union(peopleDF) \
.show()

In [ ]:
peopleDF \
.select("age", f.when(peopleDF.name == "Peter", "Piotr").otherwise(peopleDF.name).alias("name")) \
.union(peopleDF) \
.show()

In [ ]:
peopleDF \
.select(f.when(peopleDF.name == "Peter", "Piotr").otherwise(peopleDF.name).alias("name"), "age") \
.unionByName(peopleDF) \
.show()

In [ ]:
spark.sql("select age, if(name='Peter','Piotr',name) name from peopleDF union all select * from peopleDF").show()

----
- **join(other, on, how)** - zwraca nowy DF powstały na podstawie połączenia dwóch DFów w oparciu o wartości we wskazanej kolumnie/kolumnach (**T**)
- **toPandas()** - zwraca DF jako Pandas DF (**A**)
- **count()** - zwraca liczbę wierszy w DF (**A**)

In [ ]:
personDF = spark.createDataFrame([Row(surname="Wonder", name="Alice")])

peopleDF \
.join(personDF, "name") \
.toPandas()

In [ ]:
peopleDF \
.join(personDF, "name") \
.count()

In [ ]:
peopleDF \
.join(personDF, peopleDF.name == personDF.name, "outer") \
.toPandas()

In [ ]:
peopleDF \
.join(personDF, peopleDF.name == personDF.name, "outer") \
.count()

In [ ]:
personDF.createOrReplaceTempView("personDF")

spark.sql("select pl.name, pl.age, p.surname from peopleDF pl inner join personDF p on pl.name = p.name") \
.toPandas()

In [ ]:
q = "select pl.age, pl.name, p.name, p.surname from peopleDF pl full outer join personDF p on pl.name = p.name"
spark.sql(q).toPandas()

----
- **describe(cols)** - zwraca nowy DF zawierający podstawowe statystyki wszystkich lub wskazanych kolumn (**T**)

In [ ]:
peopleDF \
.describe() \
.show()

In [ ]:
peopleDF \
.describe("name") \
.show()

----
- **groupBy(cols)** - zwraca nowy DF pogrupowany po wskazanej kolumnie/kolumnach (nie jest to typowy DF, nie można go podejrzeć używając show) (**T**)
- **agg(expr)** - zwraca nowy DF powstały w wyniku zastosowania wskazanych agregacji (**T**)

In [ ]:
# max, min, avg, count, sum
empHist \
.groupBy("name") \
.max("salary") \
.show()

In [ ]:
empHist \
.groupBy("name") \
.agg(f.max("salary").alias("max_sal"), f.avg("salary").alias("avg_sal"), f.min("year").alias("first_year")) \
.show()

In [ ]:
empHist \
.agg(f.max("salary").alias("max_sal")) \
.show()

In [ ]:
spark.sql("select name, max(salary) from empHist group by name").show()

In [ ]:
q = "select name, max(salary) max_sal, avg(salary) avg_sal, min(year) first_year from empHist group by name"
spark.sql(q).show()

In [ ]:
spark.sql("select max(salary) max_sal from empHist").show()

----
- **coalesce(numPartitions)** - zwraca nowy DF z dokładnie `numPartitions` partycji, pozwala jedynie na zmniejszenie liczby partycji, unika shuffle (**T**)
- **repartition(numPartitions [,cols])** - zwraca nowy DF z dokładnie `numPartitions` partycji, wywołuje shuffle, wykorzystuje funkcję hashującą (**T**)
- **repartitionByRange(numPartitions, cols)** - zwraca nowy DF z dokładnie `numPartitions` partycji, wywołuje shuffle, wykorzystuje zakres wartości - konieczne jest wskazanie kolumny (**T**)

In [ ]:
empHist.rdd.getNumPartitions()

In [ ]:
empHist.repartition(5).rdd.glom().collect()

In [ ]:
empHist.repartition(5).rdd.glom().map(len).collect()

In [ ]:
empHist.repartition(5, "name").rdd.glom().collect()

In [ ]:
# spark.sql.shuffle.partitions
empHist.repartition("name").rdd.glom().collect()

In [ ]:
# spark.sql.shuffle.partitions
empHist.groupBy("name").max("salary").rdd.getNumPartitions()

In [ ]:
empHist.repartitionByRange(5, "name").rdd.glom().collect()

In [ ]:
empHist.repartitionByRange("name").rdd.glom().collect()

In [ ]:
empHist.repartitionByRange(5, "name").rdd.getNumPartitions()

In [ ]:
empHist.repartitionByRange(5, "name").rdd.glom().map(len).collect()

In [ ]:
spark.sql("select /*+ repartition(5) */ * from empHist").rdd.glom().collect()

In [ ]:
# od wersji 3.0.0
spark.sql("select /*+ repartition_by_range(5, name) */ * from empHist").rdd.glom().collect()

----

----

> **TODO**: Wykorzystując parametr `thresh` w dropna wyświetl DF z wierszami w których występuje maksymalnie jeden brak danych (peopleDF)

> **TODO**: Na podstawie kolumny `age` stwórz kolumnę `age_filled`, wykorzystując parametr `subset` w fillna wyświetl DF w którym braki danych w kolumnie `age_filled` zastąpione są wartością 27 (peopleDF)

> **TODO**: Na podstawie kolumny `name` stwórz kolumnę `surname`, wykorzystując parametr `subset` w replace wyświetl DF w którym wartości w kolumnie `surname` zastąpione zostały w następujący sposób: Greg -> House, Alice -> Wonder, Peter -> Parker (peopleDF)

> **TODO**: Wyświetl DF zawierający średnie, maksymalne i minimalne zarobki (zaokrąglone do pełnych wartości) we wszystkich firmach w poszczególnych latach (empHist)

----

----

### Funkcje SQL

Z funkcji tych można korzystać wewnątrz transformacji: select, withColumn, agg

In [ ]:
from pyspark.sql import functions as f

----
- **max(col)**
- **min(col)**
- **avg(col)**
- **stddev(col)**
- **variance(col)**
- **count(col)**

In [ ]:
empHist \
.agg(f.max("salary"), f.min("salary"), f.avg("salary"), 
     f.stddev("salary"), f.variance("salary"), f.count("salary")) \
.show()

In [ ]:
empHist \
.select(f.max("salary"), f.min("salary"), f.avg("salary"), 
     f.stddev("salary"), f.variance("salary"), f.count("salary")) \
.show()

In [ ]:
q = "select max(salary), min(salary), avg(salary), stddev(salary), variance(salary), count(salary) from empHist"
spark.sql(q).show()

----
Join to popularna, ale kosztowna operacja.<br>
W sytuacji, kiedy jeden z łaczonych DataFramow jest znacznie mniejszy (w szczegolnosci na tyle mały, że w całości mieści się w pamięci), zaleca sie zastosowanie broadcast hash join (mała tabela zostanie wysłana do każdego noda).<br>
W niektórych przypadkach optymalizator sam za nas zdecyduje o zastosowaniu broadcast hash join.

- **broadcast(df)** - oznacza DF jako dostatecznie mały do wykorzystania w broadcast joinie

In [ ]:
f.broadcast(personDF)
peopleDF \
.join(personDF, "name") \
.show()

In [ ]:
peopleDF \
.join(f.broadcast(personDF), "name") \
.show()

In [ ]:
q = ("select /*+ broadcast(prs) */ ppl.name, ppl.age, prs.surname from "
     "peopleDF ppl join personDF prs on ppl.name = prs.name")

spark.sql(q).show()

In [ ]:
peopleDF \
.join(f.broadcast(personDF), "name") \
.explain()

In [ ]:
q = ("select /*+ broadcast(prs) */ ppl.name, ppl.age, prs.surname from "
     "peopleDF ppl join personDF prs on ppl.name = prs.name")

spark.sql(q).explain()

----
- **lit(val)** - tworzy kolumnę ze stałą wartością
- **asc(col)/desc(col)** - pozwalają na zdefiniowanie porządku sortowania w orderBy

In [ ]:
empHist \
.withColumn("const", f.lit(12)) \
.orderBy(f.desc("year")) \
.show()

In [ ]:
spark.sql("select name, salary, year, company, 12 const from empHist order by year desc").show()

----
- **col(col)** - pozwala na odwołanie się do kolumny

In [ ]:
peopleDF.select(f.col("age").alias("AGE")).show()

In [ ]:
# do operacji na kolumnach potrzebne jest odwołanie do kolumny - nazwa nie wystarcza
peopleDF.withColumn("X", f.lit(3)).withColumn("ageX", "X" * "age").show()

In [ ]:
# peopleDF nie zawiera kolumny X
peopleDF.withColumn("X", f.lit(3)).withColumn("ageX", peopleDF["X"] * peopleDF["age"]).show()

In [ ]:
peopleDF.withColumn("X", f.lit(3)).withColumn("ageX", f.col("X") * f.col("age")).show()

In [ ]:
spark.sql("select age, name, 3 X, age * X ageX from peopleDF").show()

In [ ]:
spark.sql("select age, name, 3 X, age * 3 ageX from peopleDF").show()

------------------------------------------------------------

In [ ]:
t_df = spark.createDataFrame([('2019-04-27 09:00:00',), ('2019-05-09 12:00:00',), ('2019-06-01 16:30:00',)], 
                             ['time'])
t_df.createOrReplaceTempView("t_df")
t_df.show()
t_df.dtypes

----------------------------------------------------

----
- **split(str, pattern)** - dzieli string na podstawie wzorca (regex)
- **explode(col)** - tworzy nowy wiersz z każdego elementu arraya

In [ ]:
t_df \
.select(f.explode(f.split("time", " ")).alias("time_exploded")) \
.show()

In [ ]:
t_df \
.select(f.split("time", " ").alias("splitted")) \
.show(truncate=False)

In [ ]:
t_df \
.select(f.split("time", " ").alias("splitted")) \
.printSchema()

In [ ]:
t_df \
.select(f.split("time", " ").alias("splitted")) \
.withColumn("exploded", f.explode(f.col("splitted"))).show(truncate=False)

In [ ]:
# dwa sposoby na wyciąganie wartości (tworzenie nowych kolumn) z kolumny zawierającej array'e
t_df \
.select(f.split("time", " ").alias("splitted")) \
.withColumn("date", f.col("splitted")[0]) \
.withColumn("time", f.col("splitted").getItem(1)) \
.show(truncate=False)

In [ ]:
t_df \
.select(f.split("time", " ").alias("splitted"), f.split("time", " ")[0].alias("date"), 
        f.split("time", " ").getItem(1).alias("time")) \
.show(truncate=False)

In [ ]:
spark.sql("select explode(split(time, ' ')) time_exploded from t_df").show()

In [ ]:
spark.sql("select split(time, ' ') splitted, split(time, ' ')[0] date, split(time, ' ')[1] time from t_df") \
.show(truncate=False)

----
- **from_utc_timestamp(timestamp, tz)**

In [ ]:
t_df \
.withColumn("time", f.from_utc_timestamp("time", "GMT")) \
.show()

t_df \
.withColumn("time", f.from_utc_timestamp("time", "GMT")) \
.dtypes

In [ ]:
spark.sql("select from_utc_timestamp(time, 'GMT') time from t_df").show()

spark.sql("select from_utc_timestamp(time, 'GMT') time from t_df").dtypes

----
- **lower(col)**
- **upper(col)**
- **length(col)**
- **substring(str, pos, len)**

In [ ]:
empHist \
.select(f.lower(empHist.name), f.upper(empHist.name), f.length("name"), f.substring("name", 1, 2)) \
.show()

In [ ]:
spark.sql("select lower(name), upper(name), length(name), substring(name, 1, 2) from empHist").show()

----
- **rand(seed)**, **randn(seed)** - tworzą kolumny z losowymi wartościami z rozkładów odpowiednio jednostajnego (0-1) i normalnego (0,1)
- **when(cond, value)**

In [ ]:
peopleDF \
.select(f.rand(42), f.randn(1), 
        f.when(f.rand(42) > 0.7, "U>0.7").when(f.rand(42) > 0.4, "U>0.4").otherwise("U<=0.4").alias("U"), 
        f.when(f.randn(1) > 0.5, "N>0.5").otherwise("N<=0.5").alias("N")) \
.show()

In [ ]:
q = ("select rand(42), randn(1), "
     "case when rand(42) > 0.7 then 'U>0.7' when rand(42) > 0.4 then 'U>0.4' else 'U<=0.4' end U, "
     "case when randn(1) > 0.5 then 'N>0.5' else 'N<=0.5' end N from peopleDF")
spark.sql(q).show()

----

----

> **TODO**: Wyświetl DF z kolumną zawierającą ostatnie dwie litery imion (empHist)

> **TODO**: Wyświetl DF z kolumną zawierającą wartość `3` gdy imię ma 3 litery, `ok` gdy zarobki są wyższe niż 5000 oraz `?` w każdym innym przypadku (empHist)

----

----

### Funkcje okienne (window functions)

Służą do obliczania agregowanych wartości w grupach definiowanych oknem (window).<br>
Zwracają wiele rekordow (tyle ile na wejsciu w grupie).

In [ ]:
from pyspark.sql.window import Window

----
- **partitionBy(cols)** - konstruktor tworzący okna - podział DF ze względu na wartości w podanej kolumnie/kolumnach

In [ ]:
# definicja 'okna'
windowSpec = Window.partitionBy('name')

empHist \
.withColumn('nameCount', f.count(f.col('name')).over(windowSpec)) \
.show(30)

In [ ]:
q = "select name, salary, year, company, count(name) over (partition by name) nameCount from empHist"
spark.sql(q).show(30)

----
- **orderBy(cols)** - definiuje wewnątrz każdego okna sortowanie w oparciu o wskazaną kolumnę/kolumny
- **rank(), dense_rank(), row_number()** - zwracają ranking/numery wierszy w oknie

In [ ]:
windowSpec = Window.partitionBy("name").orderBy("year")

empHist \
.withColumn("rank", f.rank().over(windowSpec)) \
.withColumn("dense_rank", f.dense_rank().over(windowSpec)) \
.withColumn("row_num", f.row_number().over(windowSpec)) \
.show(30)

In [ ]:
q = ("select name, salary, year, company, rank() over (partition by name order by year) rank, "
     "dense_rank() over (partition by name order by year) dense_rank, "
     "row_number() over (partition by name order by year) row_num from empHist")
spark.sql(q).show(30)

----
- **rangeBetween(start, end)** - odaje wewnątrz każdego okna zakres (offset) na którym zastosowana zostanie wskazana funkcja (remisy w sortowaniu traktuje jak funkcja rank)

In [ ]:
windowSpec = Window.partitionBy("name").orderBy("year").rangeBetween(Window.unboundedPreceding,0)

empHist \
.withColumn("moving_avg", f.avg(f.col("salary")).over(windowSpec)) \
.show(30)

In [ ]:
q = ("select name, salary, year, company, avg(salary) over "
     "(partition by name order by year range between unbounded preceding and current row) moving_avg "
     "from empHist")
spark.sql(q).show(30)

- **rowsBetween(start, end)** - dodaje wewnątrz każdego okna zakres (offset) na którym zastosowana zostanie wskazana funkcja (remisy w sortowaniu traktuje jak funkcja row_number)

In [ ]:
windowSpec = Window.partitionBy("name").orderBy("year").rowsBetween(-1,1)

empHist \
.withColumn("moving_avg", f.avg(f.col("salary")).over(windowSpec)) \
.show(30)

In [ ]:
q = ("select name, salary, year, company, avg(salary) over "
     "(partition by name order by year rows between 1 preceding and 1 following) moving_avg from empHist")
spark.sql(q).show(30)

----

----

> **TODO**: Wyświetl DF z kolumną zawierającą różnicę między pensją danej osoby w konkretnym roku a średnią pensją danej osoby w analizowanym okresie (empHist)

> **TODO**: Wyświetl DF z kolumną zawierającą różnicę w pensji rok do roku dla każdej z osób - wykorzystaj funkcję `lag` (empHist)

> **TODO**: Wyświetl DF z kolumną zawierającą średnią wyliczaną z dwóch poprzednich pensji dla poszczególnych osób (empHist)

----

----

### UDF - User Defined Function

**NIEWYDAJNE**

In [ ]:
from pyspark.sql.types import IntegerType, StringType, FloatType

----
**PRZYKŁAD 1**

In [ ]:
def power3(value):
    return(value**3)

In [ ]:
power3(3)

----
- **udf(f, returnType)** - tworzy UDF zwracający kolumnę o wartościach typu `returnType`

In [ ]:
udfPower3 = f.udf(power3, FloatType())

In [ ]:
empHist.select("salary", udfPower3(f.col("salary")).alias("power3")).show()

----
- **SparkSession.udf.register(name, f, returnType)** - rejestruje UDF zwracający kolumnę o wartościach typu `returnType` z którego można korzystać w zapytaniach SQL

In [ ]:
spark.udf.register("power3", power3, FloatType())

In [ ]:
spark.sql("select salary, power3(salary) power3 from empHist").show()

----
**PRZYKŁAD 2**

In [ ]:
def divide(x,y):
    return x / y

In [ ]:
divide(10,2)

In [ ]:
udfDivide = f.udf(divide, FloatType())

In [ ]:
empHist.select(udfDivide(f.col("salary"), f.col("year")).alias("nonsens")).show()

In [ ]:
spark.udf.register("divide", divide, FloatType())

In [ ]:
spark.sql("select divide(salary, year) nonsens from empHist").show()

----

----

### ZADANIE
[Project Gutenberg](http://www.gutenberg.org/)
> Wstępnie przetwórz i zbadaj Moby Dicka
1. usuń puste linie, tytuł i nazwy rozdziałów, wynik zapisz jako `textOnlyMD`
2. policz ile słów znajduje się w tekście
3. policz ile unikalnych słów występuje w tekście
4. znajdź 10 najczęstszych słów
5. sprawdź jak często wystepowało slowo "whale"

In [ ]:
rawMD = spark.read.text("MobyDick.txt")
rawMD.show()